In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from collections import Counter

In [2]:
# VQ2版本添加 extended 经纬度
df = pd.read_csv('/mnt/d/Documents/数学建模国赛/C题目/data/GTD_vQ2.csv')
df = df.set_index('eventid')
df = df.fillna(0)

# Unknown 组织 编号 377
index_start = np.argwhere(df.index.values == 201501010001)[0][0]
index_end = np.argwhere(df.index.values == 201701010001)[0][0]
print(index_end - index_start)

df = df[index_start:]
df = df[df.gname == 377]

index_target = np.argwhere(df.index.values == 201701010005)[0][0]
print(index_target)

/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


28556
12367


In [3]:
q2_fields = ['extended','crit1','crit2','crit3','doubtterr','alternative','latitude','longitude','specificity','success','suicide','nperps','nkill','nwound','propextent','nhostkid','ransom',]
# 把country 分别表示 attacktype1 targtype1

dummies_attacktype = pd.get_dummies(df.attacktype1, prefix='attacktype')
dummies_targtype = pd.get_dummies(df.targtype1, prefix='targtype')
dummies_weaptype = pd.get_dummies(df.weaptype1, prefix='weaptype')

df = df[q2_fields]
df = pd.concat([df, dummies_attacktype, dummies_targtype, dummies_weaptype], axis=1)
df.columns

Index(['extended', 'crit1', 'crit2', 'crit3', 'doubtterr', 'alternative',
       'latitude', 'longitude', 'specificity', 'success', 'suicide', 'nperps',
       'nkill', 'nwound', 'propextent', 'nhostkid', 'ransom', 'attacktype_1',
       'attacktype_2', 'attacktype_3', 'attacktype_4', 'attacktype_5',
       'attacktype_6', 'attacktype_7', 'attacktype_8', 'attacktype_9',
       'targtype_1', 'targtype_2', 'targtype_3', 'targtype_4', 'targtype_5',
       'targtype_6', 'targtype_7', 'targtype_8', 'targtype_9', 'targtype_10',
       'targtype_11', 'targtype_12', 'targtype_13', 'targtype_14',
       'targtype_15', 'targtype_16', 'targtype_17', 'targtype_18',
       'targtype_19', 'targtype_20', 'targtype_21', 'targtype_22',
       'weaptype_2', 'weaptype_5', 'weaptype_6', 'weaptype_8', 'weaptype_9',
       'weaptype_10', 'weaptype_11', 'weaptype_12', 'weaptype_13'],
      dtype='object')

In [4]:
# 聚类
CLUSTERS=15
df_train = df[:index_target]
mbkmeans = MiniBatchKMeans(n_clusters=CLUSTERS, random_state=0).fit(df[:index_target].values)
r = mbkmeans.predict(df[:index_target].values)
print(mbkmeans.score(df[:index_target].values))
Counter(r)

-2054876.637582793


Counter({1: 4229,
         10: 430,
         14: 811,
         2: 556,
         5: 1086,
         12: 1172,
         11: 869,
         0: 797,
         6: 398,
         3: 651,
         9: 123,
         13: 371,
         4: 230,
         8: 338,
         7: 306})

In [27]:
# 聚类编号与等级编号的对应关系
df_train['level'] = r
df_train['score'] = df_train.nkill + df_train.nwound*0.3 + df_train.nhostkid*0.1
df_ana = df_train.groupby('level')[['nkill','score']].mean()
df_ana = df_ana.sort_values('score', ascending=False)
top_label = df_ana.head(5).index
top_label.values
# dict_notolevel = {0:1,1:2,3:3,4:4,2:5}
# df.level = list(map(lambda x: dict_notolevel.get(x), df.level))
# df.groupby('level')[q2_fields].mean()
# 1 0
# 2 1
# 3 3
# 4 4
# 5 2 

/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/nocater/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


array([ 4,  1, 13, 10,  2])

In [6]:
df_target = df[index_target:]
predicts = mbkmeans.transform(df_target.values)
predicts

array([[ 47.27918545,  28.31545256, 101.94360126, ...,  43.05973224,
         71.80392292,  21.67580179],
       [106.62158292, 101.36355484,  45.02569517, ..., 100.36884109,
        108.34830928, 118.41581939],
       [ 45.45903445,  27.61119789, 101.92979494, ...,  42.18378797,
         70.32234867,  21.47098603],
       ...,
       [ 43.28531707,  21.64804242, 102.33536431, ...,  36.67294862,
         66.31526529,  28.1108099 ],
       [ 62.32901274,  51.59920333, 104.27757931, ...,  66.05955218,
         90.44013964,   5.22893706],
       [ 89.32605197,  85.34005066, 116.67174955, ...,  99.2365813 ,
        118.87949062,  38.27613108]])

In [12]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def func_sort(x):
    index = np.argsort(x)
    l = [0 for i in range(CLUSTERS)]
    for e,i in enumerate(index):
        l[i] = int(e+1)
    return l

In [38]:
index_target = [201701090031, 201702210037, 201703120023, 201705050009, 201705050010, 201707010028, 201707020006, 201708110018, 201711010006, 201712010003]
df_target = df.loc[index_target]
print(df_target.index)
print()

distances = mbkmeans.transform(df_target.values)
df_result = pd.DataFrame(distances, columns=list(range(CLUSTERS)), index=index_target)
print(df_result)
df_result = df_result.apply(func_sort, axis=1)
# df_result = pd.DataFrame(df_result, columns=list(range(CLUSTERS)))

list_target = []
for row in df_result.values:
    row = np.array(row)
    print(list(top_label.values))
    list_target.append(row[list(top_label.values)])
    
list_target

Int64Index([201701090031, 201702210037, 201703120023, 201705050009,
            201705050010, 201707010028, 201707020006, 201708110018,
            201711010006, 201712010003],
           dtype='int64', name='eventid')

                      0           1           2           3           4   \
201701090031   34.464222   14.346861  106.837458   76.533385  113.466653   
201702210037   45.682957   28.275431  103.803318   57.582757  119.373535   
201703120023   59.129923   63.309003  126.711246  101.366743  121.140162   
201705050009   15.818505   34.173064  113.542374   82.640128  107.023357   
201705050010   15.430833   34.068031  113.487934   82.558791  106.969487   
201707010028   21.249546   39.146800  118.151307   94.802805  106.696598   
201707020006  106.487280  102.535010   42.853231  131.075068   41.716284   
201708110018   45.244072   24.754509  103.026151   58.366006  119.260509   
201711010006   54.213871   37.410352  106.152876   60.579274  123.237820   
201712010003   39.42

[array([13,  1,  8, 11, 12]),
 array([14,  4,  9, 12, 11]),
 array([12,  2,  3, 11, 13]),
 array([11,  4,  2, 12, 13]),
 array([11,  4,  2, 12, 13]),
 array([11,  4,  1, 13, 14]),
 array([ 2,  5, 12,  1,  3]),
 array([14,  4, 10, 12, 11]),
 array([14,  4,  9, 12, 11]),
 array([13,  1,  8, 11, 12])]

In [26]:
a = np.array([106.760092  , 37.751814  , 40.569300   ,15.118342 , 113.291522])
a[[1,2]]


array([37.751814, 40.5693  ])